In [2]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.sparse
from matplotlib import cm
import matplotlib as mpl
from cycler import cycler
from matplotlib import rc
import scipy.stats
from sklearn.metrics import explained_variance_score
from tqdm import tnrange

In [3]:
true_diff = np.load("/scratch/yjk27/CA1_clust4-60/data/V_diff_stimseed1_set1.npy")[-50000*20:].reshape(20,50000)
one_diff_test = np.load("/scratch/yjk27/CA1_clust4-60/hand/glm_s10_h1_output.npz")["test"]
two_diff_test = np.load("/scratch/yjk27/CA1_clust4-60/hand/glm_s10_h2_output.npz")["test"]
gru_diff_test = np.load("/scratch/yjk27/CA1_clust4-60/hand/gru_s10_h20_output.npz")["test"]
gru_dend_diff_test = np.load("/scratch/yjk27/CA1_clust4-60/hand/gru_dend_s5_h20_output.npz")["test"]

In [4]:
var_exp = np.zeros((20,4))

for i in range(20):
    var_exp[i,0] = explained_variance_score(true_diff[i], one_diff_test[i])
    var_exp[i,1] = explained_variance_score(true_diff[i], two_diff_test[i])
    var_exp[i,2] = explained_variance_score(true_diff[i], gru_diff_test[i])
    var_exp[i,3] = explained_variance_score(true_diff[i], gru_dend_diff_test[i])

In [ ]:
mpl.rcParams['pdf.fonttype'] = 42
mpl.rcParams['font.sans-serif'] = "Arial"
mpl.rcParams['font.family'] = "sans-serif"
plt.rcParams['font.size'] = '13'

fig = plt.figure(constrained_layout=True, figsize = (5,5))
gs = fig.add_gridspec(1,1, wspace=0, hspace=0)
ax = fig.add_subplot(gs[0,0])

colors = ["green", "blue", "red", "purple"]
cap_colors = ["green", "green", "blue", "blue", "red", "red", "purple", "purple"]

bp = ax.boxplot(var_exp, positions = [1,2,3,4], widths = 0.6, patch_artist=True)
for i in range(var_exp.shape[1]):
    plt.setp(bp['boxes'][i], facecolor=colors[i])
    plt.setp(bp['boxes'][i], alpha=0.5)
    plt.setp(bp["fliers"][i], markeredgecolor=colors[i])
    plt.setp(bp["boxes"][i], color=colors[i])
    plt.setp(bp["medians"][i], color=colors[i])
for i in range(var_exp.shape[1]*2):
    plt.setp(bp['whiskers'][i], color=cap_colors[i])
    plt.setp(bp['caps'][i], color=cap_colors[i])
    

    
ax.set_xticklabels(["static", "static w/\nmultiplex", "dynamic", "dynamic w/\ndendritic\ninteractions"])
ax.set_ylim(0,1)
ax.margins(x=0.005, y=0.005)
ax.spines["right"].set_visible(False)
ax.spines["top"].set_visible(False)
ax.spines["left"].set_bounds((0,1))
ax.spines["bottom"].set_visible(False)
ax.tick_params(axis="x", length=0)
ax.set_ylabel("variance explained")
 
"""
#ax5.set_xticklabels(["4", "8", "12"])
#ax5.set_xlabel("number of clusters")
ax5.set_ylim(0,1)
ax5.margins(x=0.005, y=0.005)
ax5.spines.right.set_visible(False)
ax5.spines.top.set_visible(False)
ax5.set_title("$\mathregular{v_{Na}}$")
ax5.spines.bottom.set_visible(False)
ax5.tick_params(axis="x", length=0)
ax5.set_yticks(np.arange(0,1.01,0.5))
ax5.spines.left.set_bounds((0,1))
"""

#plt.show()
#fig.savefig("/scratch/yjk27/dendrite/fig_dend_interact/fig_dend_interact_raw.pdf", bbox_inches="tight", transparent=True)